In [1]:
import warnings
warnings.filterwarnings("ignore")
import pymc4 as pm4
import pymc3 as pm3
import tensorflow as tf
import numpy as np
import arviz as az

W0902 21:55:53.908671 140667116451456 blas_headers.py:988] Using NumPy C-API based implementation for BLAS functions.


```
Original Stan model
data {
  int<lower=0> J; // number of schools
  real y[J]; // estimated treatment effects
  real<lower=0> sigma[J]; // s.e. of effect estimates
}
parameters {
  real mu;
  real<lower=0> tau;
  real eta[J];
}
transformed parameters {
  real theta[J];
  for (j in 1:J)
    theta[j] <- mu + tau * eta[j];
}
model {
  eta ~ normal(0, 1);
  y ~ normal(theta, sigma);
}
```

In [2]:
J = 8
y = np.array([28,  8, -3,  7, -1,  1, 18, 12], dtype=np.float32)
sigma = np.array([15, 10, 16, 11,  9, 11, 10, 18], dtype=np.float32)

In [3]:
with pm3.Model() as schools:

    eta = pm3.Normal('eta', 0, 1, shape=J)
    mu = pm3.Normal('mu', 0, sigma=1e6)
    tau = pm3.HalfCauchy('tau', 25)

    theta = mu + tau * eta

    obs = pm3.Normal('obs', theta, sigma=sigma, observed=y)
    #trace = pm3.sample()

In [4]:
@pm4.model
def schools_pm4():
    eta = yield pm4.Normal("eta", tf.zeros(J), tf.ones(J))
    mu = yield pm4.Normal("mu", 0., sigma=1e6)

    tau = yield pm4.HalfNormal('tau', 25.)

    theta = mu + tau * eta

    obs = yield pm4.Normal('obs', theta, sigma=sigma, observed=y)
    

In [5]:
pm4.evaluate_model_transformed(schools_pm4())

(None, SamplingState(
     untransformed_values: ['schools_pm4/eta', 'schools_pm4/mu', 'schools_pm4/tau']
     transformed_values: ['schools_pm4/__log_tau']
     observed_values: ['schools_pm4/obs']
     distributions: ['Normal:schools_pm4/eta', 'Normal:schools_pm4/mu', 'HalfNormal:schools_pm4/tau', 'Normal:schools_pm4/obs']
     num_potentials=1))

In [7]:
tf_trace = pm4.inference.sampling.sample(schools_pm4(),)

KeyboardInterrupt: 